**Created by: ayoub touti, mohamed aziz catalan**

# **data scraping from imdb**

In [11]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from textblob import TextBlob
from requests import get
from time import sleep
from random import randint
from warnings import warn




In [2]:
pages = np.arange(1, 20, 50) 
headers = {'Accept-Language': 'en-US,en;q=0.8'} # If this is not specified, the default language is Mandarin

In [3]:
#initialize empty lists to store the variables scraped
titles = []
years = []
ratings = []
genres = []
runtimes = []
imdb_ratings = []
imdb_ratings_standardized = []
metascores = []
votes = []


In [12]:
for page in pages:
    
    #get request
    response = get("https://www.imdb.com/search/title?genres=sci-fi&" 
                   + "start=" 
                   + str(page) 
                   + "&explore=title_type,genres&ref_=adv_prv")
    
    sleep(randint(8,15))
     
    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    #parse the content of current iteration of request
    page_html = BeautifulSoup(response.text, 'html.parser')
        
    movie_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
    
    #extract the 50 movies for that page
    for container in movie_containers:

        #conditional for all with metascore
        if container.find('div', class_ = 'ratings-metascore') is not None:

            #title
            title = container.h3.a.text
            titles.append(title)

            #year released
            year = container.h3.find('span', class_= 'lister-item-year text-muted unbold').text
            years.append(year)

            #rating
            rating = container.p.find('span', class_= 'certificate').text
            ratings.append(rating)

            #genre
            genre = container.p.find('span', class_ = 'genre').text
            genres.append(genre)

            #runtime
            time = container.p.find('span', class_ = 'runtime').text
            runtimes.append(time)

            #IMDB ratings
            imdb = float(container.strong.text)
            imdb_ratings.append(imdb)

            #Metascore
            m_score = container.find('span', class_ = 'metascore').text
            metascores.append(int(m_score))

            #Number of votes
            vote = container.find('span', attrs = {'name':'nv'})['data-value']
            votes.append(int(vote))

In [13]:
sci_fi_df = pd.DataFrame({'movie': titles,
                      'year': years,
                      'rating': ratings,
                      'genre': genres,
                      'runtime_min': runtimes,
                      'imdb': imdb_ratings,
                      'metascore': metascores,
                      'votes': votes}
                      )

sci_fi_df.loc[:, 'year'] = sci_fi_df['year'].str[-5:-1] # two more data transformations after scraping
# Drop 'ovie' bug
# Make year an int
sci_fi_df['n_imdb'] = sci_fi_df['imdb'] * 10
final_df = sci_fi_df.loc[sci_fi_df['year'] != 'ovie'] # One small issue with the scrape on these two movies so just dropping those ones.
final_df.loc[:, 'year'] = pd.to_numeric(final_df['year'])

In [ ]:
sci_fi_df

# **Reviews**

In [15]:
page = requests.get("https://www.imdb.com/title/tt6443346/reviews?ref_=tt_ov_rt")
soup = BeautifulSoup(page.content, "html.parser")
rev = soup.findAll('div', class_='text show-more__control')
movies = []
for movie in rev:
    movies.append(movie.text)
df_subset = pd.DataFrame()
df_subset['Black Adam']=movies

In [16]:
df_subset

,Black Adam
0,Was this script the result of a 5th grade clas...
1,Is there ever gonna be a character arc? Terrib...
2,"I hoped ""Black Adam"" would be a decent entry i..."
3,Thought Bland Adam was another misfire for the...
4,Black Adam is possibly one of the most generic...
5,"Black Adam is not without its flaws, but i sti..."
6,Tragic cinema for brainless people. They even ...
7,Black Adam is the perfect example of how inacc...
8,I was shocked by how much I disliked Black Ada...
9,In the superhero movie genre I would rank this...


# **Sentiments**

In [28]:
df = pd.DataFrame()

In [29]:
df['Reviews of people']=movies

In [30]:
list=[]
for i in range(len(movies)):
    text=TextBlob(movies[i])
    x=text.sentiment.polarity # x between -1 and 1
    list.append(round(x,2))

In [31]:
sent=[]
j=[]
for j in list:
    if j>0:
        x="positive"
    elif j==0:
        x="neutre"
    else :
        x="negative"
    sent.append(x) 

In [32]:
df['sentiment']=sent

In [33]:
df

,Reviews of people,sentiment
0,Was this script the result of a 5th grade clas...,negative
1,Is there ever gonna be a character arc? Terrib...,negative
2,"I hoped ""Black Adam"" would be a decent entry i...",positive
3,Thought Bland Adam was another misfire for the...,negative
4,Black Adam is possibly one of the most generic...,positive
5,"Black Adam is not without its flaws, but i sti...",positive
6,Tragic cinema for brainless people. They even ...,negative
7,Black Adam is the perfect example of how inacc...,positive
8,I was shocked by how much I disliked Black Ada...,negative
9,In the superhero movie genre I would rank this...,positive
